In [1]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

c:\Users\danny\miniconda3\envs\azure_tuto\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:305: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


In [4]:
frontdesk_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.""",
)

concierge_agent = AssistantAgent(
    "concierge_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, respond with 'APPROVE'
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """,
)

In [5]:
termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat(
    [frontdesk_agent, concierge_agent], termination_condition=termination
)

async for message in team.run_stream(task="I would like to plan a trip to Paris."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='I would like to plan a trip to Paris.' type='TextMessage'
source='planner_agent' models_usage=RequestUsage(prompt_tokens=100, completion_tokens=11) metadata={} content='Visit the Louvre Museum for an unforgettable art experience.' type='TextMessage'
source='concierge_agent' models_usage=RequestUsage(prompt_tokens=112, completion_tokens=92) metadata={} content="While the Louvre is undoubtedly a remarkable museum, it's one of the most visited and tourist-heavy locations in Paris. To create a more authentic experience for travelers, I recommend suggesting a visit to local galleries or smaller, lesser-known museums that showcase contemporary art or focus on specific themes related to Parisian culture. Also, consider recommending exploration of various neighborhoods or art districts where travelers can interact with local artists and artisans directly, making for a more genuine Parisian experience." type='TextMessage'
source='planner_agen